<참조 : https://www.tensorflow.org/tutorials/load_data/csv?hl=ko>

### 01. 설정
  - Titanic 승객 목록

In [1]:
import functools

import numpy as np
import tensorflow as tf

In [2]:
TRAIN_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"

train_file_path = tf.keras.utils.get_file('train.csv', TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file('eval.csv', TEST_DATA_URL)

16384/13049 [=====================================] - 1s 40us/step


In [3]:
# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)

#### set_printoptions
  - https://runebook.dev/ko/docs/numpy/reference/generated/numpy.set_printoptions

### 02. 데이터 로드하기

In [16]:
# head {train_file_path}
# 오류 발생

In [17]:
LABEL_COLUMN = 'survived'
LABELS = [0, 1]

In [19]:
def get_dataset(file_path, **kwargs):
    dataset = tf.data.experimental.make_csv_dataset(
        file_path,
        batch_size=5,     # Artificially small to make examples easier to show.
        label_name=LABEL_COLUMN,
        na_value='?',
        num_epochs=1,
        ignore_errors=True,
        **kwargs
    )
    
    return dataset

raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)

In [23]:
def show_batch(dataset):
    for batch, label in dataset.take(1):
        for key, value in batch.items():
            print('{:20s}: {}'.format(key, value.numpy()))

In [24]:
show_batch(raw_train_data)

sex                 : [b'female' b'female' b'male' b'female' b'male']
age                 : [18. 23. 25. 16. 34.]
n_siblings_spouses  : [0 1 1 0 1]
parch               : [1 0 0 0 0]
fare                : [ 23.    113.275  26.      7.733  21.   ]
class               : [b'Second' b'First' b'Second' b'Third' b'Second']
deck                : [b'unknown' b'D' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Cherbourg' b'Southampton' b'Queenstown' b'Southampton']
alone               : [b'n' b'n' b'n' b'y' b'n']


In [26]:
CSV_COLUMNS = ['survived', 'sex', 'age', 'n_siblings_spouses', 'parch',
              'fare', 'class', 'deck', 'embark_town', 'alone']

temp_dataset = get_dataset(train_file_path, column_names=CSV_COLUMNS)

show_batch(temp_dataset)

sex                 : [b'male' b'male' b'male' b'female' b'female']
age                 : [32.  6. 47. 28. 16.]
n_siblings_spouses  : [0 0 0 0 0]
parch               : [0 1 0 0 1]
fare                : [10.5   12.475 34.021  8.05  39.4  ]
class               : [b'Second' b'Third' b'First' b'Third' b'First']
deck                : [b'unknown' b'E' b'D' b'unknown' b'D']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Southampton'
 b'Southampton']
alone               : [b'y' b'n' b'y' b'y' b'n']


In [28]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'class', 'deck', 'alone']

temp_dataset = get_dataset(train_file_path, select_columns=SELECT_COLUMNS)

show_batch(temp_dataset)

age                 : [28. 19.  5. 35. 35.]
n_siblings_spouses  : [0 3 2 0 0]
class               : [b'First' b'First' b'Third' b'Second' b'First']
deck                : [b'C' b'C' b'unknown' b'unknown' b'E']
alone               : [b'y' b'n' b'n' b'y' b'y']


### 03. 데이터 전처리

#### 03. 01. 연속 데이터
  - 데이터가 이미 적절한 숫자 형식인 경우
    - 데이터를 모델로 전달하기 전에 벡터로 묶을 수 있습니다

In [30]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'parch', 'fare']
DEFAULTS = [0, 0.0, 0.0, 0.0, 0.0]
temp_dataset = get_dataset(train_file_path,
                          select_columns=SELECT_COLUMNS,
                          column_defaults=DEFAULTS)
show_batch(temp_dataset)

age                 : [42. 22. 30. 21. 33.]
n_siblings_spouses  : [1. 0. 1. 0. 0.]
parch               : [0. 0. 0. 0. 2.]
fare                : [26.     7.25  24.    77.958 26.   ]


In [31]:
example_batch, labels_batch = next(iter(temp_dataset))

In [32]:
# 모든 열을 묶는 함수
def pack(features, label):
    return tf.stack(list(features.values()), axis=-1), label

In [33]:
# pack 함수를 데이터세트의 각 요소에 적용
packed_dataset = temp_dataset.map(pack)

for features, labels in packed_dataset.take(1):
    print(features.numpy())
    print()
    print(labels.numpy())

[[ 27.      0.      0.      8.663]
 [ 45.      1.      1.    164.867]
 [ 36.      1.      1.     24.15 ]
 [ 24.      0.      0.     79.2  ]
 [ 28.      0.      0.      7.879]]

[1 1 0 0 1]


In [34]:
# 혼합 데이터세트로 다시 전환합니다
show_batch(raw_train_data)

sex                 : [b'male' b'male' b'female' b'male' b'female']
age                 : [30. 20. 35. 39. 39.]
n_siblings_spouses  : [1 1 1 1 1]
parch               : [0 1 0 5 5]
fare                : [24.    15.742 53.1   31.275 31.275]
class               : [b'Second' b'Third' b'First' b'Third' b'Third']
deck                : [b'unknown' b'unknown' b'C' b'unknown' b'unknown']
embark_town         : [b'Cherbourg' b'Cherbourg' b'Southampton' b'Southampton' b'Southampton']
alone               : [b'n' b'n' b'n' b'n' b'n']


In [35]:
example_batch, labels_batch = next(iter(temp_dataset))

- 숫자 특성 목록을 선택 -> 단일 열로 묶는 전처리

In [36]:
class PackNumericFeatures(object):
    def __init__(self, names):
        self.names = names
        
    def __call__(self, features, labels):
        numeric_features = [features.pop(name) for name in self.names]
        numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_features]
        numeric_features = tf.stack(numeric_features, axis=-1)
        features['numeric'] = numeric_features
        
        return features, labels

In [37]:
NUMERIC_FEATURES = ['age', 'n_siblings_spouses', 'parch', 'fare']

packed_train_data = raw_train_data.map(PackNumericFeatures(NUMERIC_FEATURES))

packed_test_data = raw_test_data.map(PackNumericFeatures(NUMERIC_FEATURES))

In [38]:
show_batch(packed_train_data)

sex                 : [b'male' b'female' b'male' b'male' b'male']
class               : [b'Third' b'Third' b'Third' b'Third' b'Third']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Cherbourg' b'Southampton' b'Southampton' b'Southampton']
alone               : [b'y' b'y' b'y' b'y' b'y']
numeric             : [[28.     0.     0.     7.55 ]
 [28.     0.     0.     7.229]
 [19.     0.     0.     7.896]
 [35.     0.     0.     7.05 ]
 [25.     0.     0.     7.05 ]]


In [39]:
example_batch, labels_batch = next(iter(packed_train_data))

#### 데이터 정규화
  - 연속 데이터는 항상 정규화되어야 합니다

In [40]:
import pandas as pd
desc = pd.read_csv(train_file_path)[NUMERIC_FEATURES].describe()
desc

,age,n_siblings_spouses,parch,fare
count,627.000000,627.000000,627.000000,627.000000
mean,29.631308,0.545455,0.379585,34.385399
std,12.511818,1.151090,0.792999,54.597730
min,0.750000,0.000000,0.000000,0.000000
25%,23.000000,0.000000,0.000000,7.895800
50%,28.000000,0.000000,0.000000,15.045800
75%,35.000000,1.000000,0.000000,31.387500
max,80.000000,8.000000,5.000000,512.329200


In [41]:
MEAN = np.array(desc.T['mean'])
STD = np.array(desc.T['std'])

In [42]:
def normalize_numeric_data(data, mean, std):
    # Center the data
    return (data-mean)/std

- 숫자 열을 만듭니다
  - tf.feature_columns.numeric_column API : 각 배치에서 실행될 normalizer_fn 인수를 허용합니다
  - functools.partial를 사용하여 MEAN / STD를 정규화 fn에 바인딩 합니다

In [43]:
# See what you just created.
normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD)

numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalizer,
                                                 shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]
numeric_column

NumericColumn(key='numeric', shape=(4,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function normalize_numeric_data at 0x000000E77D194940>, mean=array([29.631,  0.545,  0.38 , 34.385]), std=array([12.512,  1.151,  0.793, 54.598])))

- 모델을 훈련할 때 이 특성 열을 포함하여 숫자 데이터 블록을 선택하고 중앙에 배치합니다

In [44]:
example_batch['numeric']

<tf.Tensor: shape=(5, 4), dtype=float32, numpy=
array([[28.   ,  8.   ,  2.   , 69.55 ],
       [49.   ,  0.   ,  0.   ,  0.   ],
       [49.   ,  1.   ,  0.   , 76.729],
       [33.   ,  0.   ,  2.   , 26.   ],
       [28.   ,  0.   ,  0.   ,  7.829]], dtype=float32)>

In [45]:
numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)
numeric_layer(example_batch).numpy()

array([[-0.13 ,  6.476,  2.043,  0.644],
       [ 1.548, -0.474, -0.479, -0.63 ],
       [ 1.548,  0.395, -0.479,  0.776],
       [ 0.269, -0.474,  2.043, -0.154],
       [-0.13 , -0.474, -0.479, -0.486]], dtype=float32)

#### 03. 02. 범주형 데이터
  - CSV 데이터의 일부 열은 범주형 데이터 : 컨텐츠는 제한된 옵션 세트 중 하나입니다
  - tf.feature_column API를 사용하여 각 범주 열에 대해 tf.feature_column.indicator_column을 가진 모음을 작성합니다

In [46]:
CATEGORIES = {
    'sex': ['male', 'female'],
    'class': ['First', 'Second', 'Third'],
    'deck': ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    'embark_town': ['Cherbourg', 'Southhampton', 'Queenstown'],
    'alone': ['y', 'n']
}

In [47]:
categorical_columns = []

for feature, vocab in CATEGORIES.items():
    cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
    categorical_columns.append(tf.feature_column.indicator_column(cat_col))

In [48]:
# See what you just created.
categorical_columns

[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='class', vocabulary_list=('First', 'Second', 'Third'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='deck', vocabulary_list=('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Cherbourg', 'Southhampton', 'Queenstown'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='alone', vocabulary_list=('y', 'n'), dtype=tf.string, default_value=-1, num_oov_buckets=0))]

In [49]:
categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)
print(categorical_layer(example_batch).numpy()[0])

[0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]


#### 03. 03. 결합된 전처리 레이어

In [50]:
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numeric_columns)

In [51]:
print(preprocessing_layer(example_batch).numpy()[0])

[ 0.     1.     0.     0.     1.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.     0.     0.    -0.13   6.476
  2.043  0.644  0.     1.   ]


### 04.  모델 빌더

In [52]:
model = tf.keras.Sequential([
    preprocessing_layer,
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1)
])

In [53]:
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy']
)

### 05. 훈련, 평가 및 예측하기

In [54]:
train_data = packed_train_data.shuffle(500)
test_data = packed_test_data

In [55]:
model.fit(train_data, epochs=20)

Epoch 1/20
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
126/126 [==============================] - 10s 35ms/step - loss: 0.4948 - accuracy: 0.7496
Epoch 2/20
126/126 [==============================] - 1s 5ms/step - loss: 0.4211 - accuracy: 0.8150
Epoch 3/20
126/126 [==============================] - 1s 4ms/step - loss: 0.3945 - accuracy: 0.8246
Epoch 4/20
126/126 [==============================] - 1s 4ms/step - loss: 0.3871 - accuracy: 0.8389
Epoch 5/20
126/126 [==============================] - 1s 5ms/step - loss: 0.3759 - accuracy: 0.8373
Epoch 6/20
126/126 [==============================] - 1s 5ms/step - loss: 0.3668 - accuracy: 0.8341
Epoch 7/20
126/126 [==============================] - 1s 6ms/step - loss: 0.3633 - accuracy: 0.8421
Epoch 8/20
126/126 [==============================] - 1s 5ms/step - loss: 0.3543 - accuracy: 0.8469
Epoch 9/20
126/126 [==============================] - 1s 5ms/step - loss: 0.3479 - accur

In [56]:
test_loss, test_accuracy = model.evaluate(test_data)

print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

Consider rewriting this model with the Functional API.
53/53 [==============================] - 1s 5ms/step - loss: 0.4417 - accuracy: 0.8485


Test Loss 0.44168105721473694, Test Accuracy 0.8484848737716675


In [57]:
predictions = model.predict(test_data)

# Show some results
for prediction, survived in zip(predictions[:10], list(test_data)[0][1][:10]):
    prediction = tf.sigmoid(prediction).numpy()
    print('Predicted survival: {:.2%}'.format(prediction[0]), '  |  Actual outcome: ',
         ('SURVIVED' if bool(survived) else 'DIED'))

Consider rewriting this model with the Functional API.
Predicted survival: 81.25%   |  Actual outcome:  SURVIVED
Predicted survival: 26.13%   |  Actual outcome:  DIED
Predicted survival: 80.66%   |  Actual outcome:  DIED
Predicted survival: 52.77%   |  Actual outcome:  DIED
Predicted survival: 14.80%   |  Actual outcome:  DIED
